In [1]:
from dataset_UNSW_NB15 import Dataset_UNSW_NB15
from dataset_UNSW_NB15_1 import Dataset_UNSW_NB15_1
from dataset_NSL_KDD import Dataset_NSL_KDD
from dataset_IoT_Network_Intrusion_Dataset import Dataset_IoT_Network_Intrusion_Dataset
from experiment import Experiment
import matplotlib.pyplot as plt


In [ ]:
dataset = Dataset_UNSW_NB15(10000)
X_train, X_test, y_train, y_test, config = dataset.get_data()
exp = Experiment(X_train, X_test, y_train, y_test, config)
exp.run(k=500, n_fi=50, n_pca=1)

In [2]:
dataset = Dataset_UNSW_NB15_1(10000)
X_train, X_test, y_train, y_test, config = dataset.get_data()
exp = Experiment(X_train, X_test, y_train, y_test, config)
exp.run(k=500, n_fi=1, n_pca=90)

/Users/ham/codes/IDS_IoT/dataset_UNSW_NB15_1.py:64: DtypeWarning: Columns (1,3,47) have mixed types. Specify dtype option on import or set low_memory=False.
  self.data = pd.read_csv(self.DATA_FILE_PATH)


Invalid dsport: 0


In [ ]:
dataset = Dataset_NSL_KDD(10000)
X_train, X_test, y_train, y_test, config = dataset.get_data()
exp = Experiment(X_train, X_test, y_train, y_test, config)
exp.run(k=500, n_fi=1, n_pca=90)

In [ ]:
dataset = Dataset_IoT_Network_Intrusion_Dataset(10000)
X_train, X_test, y_train, y_test, config = dataset.get_data()
exp = Experiment(X_train, X_test, y_train, y_test, config)
exp.run(k=500, n_fi=1, n_pca=90)